In [1]:
import os
from pypdf2 import PdfReader

def prepare_dataset(data_path):
    dataset = []
    for filename in os.listdir(data_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(data_path, filename)
            reader = PdfReader(pdf_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            dataset.append({"input": text, "labels": text})  # Assuming you want to use the same text for input and labels
    return dataset

ModuleNotFoundError: No module named 'pypdf2'

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from torch.quantization import quantize_dynamic

# Load pre-trained model and tokenizer
model_name = "meta-llama/Llama-2-7b"
model = LlamaForCausalLM.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Quantize the model (post-training quantization)
model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

# Fine-tuning loop (simplified)
def fine_tune(model, tokenizer, dataset):
    model.train()
    for epoch in range(num_epochs):
        for batch in dataset:
            inputs = tokenizer(batch['input'], return_tensors="pt", padding=True, truncation=True)
            labels = tokenizer(batch['labels'], return_tensors="pt", padding=True, truncation=True)
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

# Define hyperparameters
num_epochs = 3
learning_rate = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Load and prepare your dataset
data_path = "PDFS"
dataset = prepare_dataset(data_path)

# Fine-tune the model
fine_tune(model, tokenizer, dataset)

# Save the fine-tuned model
model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")
